In [1]:
import pandas as pd
import numpy as np
import os

In [2]:

def fn_knn(df_p, X, y, target):
    
    df_p = df_p.iloc[: , :-1]
      
    # Seperating numerical and categorical features
    X_num = X.select_dtypes(include='number')
    X_cat = X.select_dtypes(include='object')
    
    # Scaling numerical features
    s = MinMaxScaler()
    X_num_scaled = s.fit_transform(X_num)
    X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
    
    # Label encoding on categotical features
    # Label encoder is accepting only 1d array, so I'm label encoding for each column and storing it in a dictionary
    X_encoded = pd.DataFrame({})
    my_le = {}
    for item in X_cat.columns:
        my_le[item] = LabelEncoder()
        X_encoded[item] = my_le[item].fit_transform(X_cat[item]) 
        
    # Concat scaled numerical and encoded categorical features
    X = pd.concat([X_encoded, X_num_scaled], axis=1) 
    
    # Seperating numerical and categorical of predication data set
    df_p_num = df_p.select_dtypes(include='number')
    df_p_cat = df_p.select_dtypes(include='object')

    # Scaling numerical features of prediction data set
    df_p_num_scaled = s.transform(df_p_num)
    df_p_num_scaled = pd.DataFrame(df_p_num_scaled, columns=df_p_num.columns)
    
    # Scaling categorical features of prediction data set
    # making sure that if male is taken as 1 in the earlier label encoding, the same (male=1) is taken now
    df_p_cat_encoded = pd.DataFrame({})
    for item in df_p_cat.columns:
        df_p_cat_encoded[item] = my_le[item].transform(df_p_cat[item]) 
        
    # Concat scaled numerical and encoded categorical columns of prediction data
    X_pred = pd.concat([df_p_cat_encoded, df_p_num_scaled], axis=1)
    
    # Find the eucledian distance between each row of prediction data and original data set
    total = []
    b = np.array(X_pred.iloc[0, : ]) # first row of prediction data is converted into an array
    for item in X.index: # iterate through each row of original data set and convert it to an numpy array as well
        dist = np.array(X.iloc[item, : ]) # find the distance between the above 2 (numpy row of prediction data and numpy row of original data)
        summ = np.square(b - dist).sum()  # find the eucleadian distance and sum it up
        total.append(summ) # append the distances to a list (which will be later added as a column)

    X['distance'] = total
    
    # concat the target column(y) with original data set features
    X = pd.concat([X, y], axis=1) 
    
    # Sort the data according to distance so that the nearest one is at the top
    # Group them by disance so that the nearest ones(at same distances) could be clustered together
    # The predicted pay will be the 
    
    X = X.sort_values(by='distance').groupby(by='distance')[target].mean().reset_index()
    
    # Return the first row of target column
    return X.iloc[0,1]


# Creating dictionaries for original dataset and prediction data. 
# This could be modified to take directly from files as well

# d_test = {'Degree':['BA', 'MA', 'DS','DS'],
#           'Gender':['m','f','m','m'],
#           'cg':[3,4,5,5],
#           'cg1':[6,7,8,8],
#           'Pay':[200,300,400,600]  
#          }
 
# df_pred= {'Degree':['DS','MA'],
#         'Gender':['m','f'],
#           'cg': [5,4],
#           'cg1': [8,7]
#         }

# df = pd.DataFrame(d_test)
# df_pred = pd.DataFrame(df_pred)

df = pd.read_csv("CO2_Emissions.csv")
df_pred = pd.read_csv("CO2_prediction.csv")


ml = []
X = df.iloc[ : , :-1] # Features (all columns except last column)
y = df.iloc[ : , -1]  # Target (last column)

# Checking if the columns are in same order in both original and prediction data set
ml1 = [item for item in df.columns]
ml2 = [item for item in df_pred.columns]
# print(list(zip(ml1,ml2)))

target = ml2[::-1][0] # this will be used when we have to group by and get the avergae of our target column

# Passing each row of prediction data to a function which will return the predicted value. 
# The predicted values are appended to a list and added as a new column
for item in df_pred.index:
    k = df_pred.iloc[item:item+1, : ]
    g = fn_knn(k, X, y, target)
    ml.append(g)     

df_pred[f"{target}_predicted"]  = ml 
df_pred

FileNotFoundError: [Errno 2] No such file or directory: 'CO2_Emissions.csv'